# Жанровая классификация музыки

In [68]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras

## Извлечение информации из мелодий

10 жанров. Каждый жанр: 100 мелодий. Всего: 1000 аудиофайлов.

## Построение спектрограммы для каждого аудиофайла

In [32]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./musicdb/genres/{g}'):
        songname = f'./musicdb/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

## Извлечение характеристик из спектрограммы

### 1. Записывание информации в файл

In [35]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./musicdb/genres/{g}'):
        songname = f'./musicdb/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

### 2. Анализ данных

In [36]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,42.351029,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.wav,0.340983,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,35.874684,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,26.871927,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,22.153301,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [38]:
# удаление ненужной информации
data = data.drop(['filename'],axis=1)

In [39]:
data

,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.349943,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,42.351029,-6.376457,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,0.340983,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,35.874684,2.916038,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,-13.987036,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,0.404779,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,26.871927,1.754463,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,0.308590,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,22.153301,-32.489270,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.351991,2008.581132,2106.617024,4254.215942,0.089267,-153.632304,109.857258,-23.085707,59.040677,-7.490499,...,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717573,-1.189238,rock
996,0.398653,2006.051164,2068.327905,4147.374921,0.097659,-142.424217,116.219780,-32.177077,49.122974,-8.375371,...,14.065346,-16.394693,4.152589,-4.350760,3.736455,-10.845639,1.875218,-7.459579,-2.802677,rock
997,0.432103,2077.190361,1926.989678,4030.767293,0.121824,-125.031313,115.194965,-47.993506,52.834532,-13.480977,...,10.481961,-17.951917,1.211113,-11.534863,1.844773,-12.847902,3.447424,-12.594178,-2.107003,rock
998,0.362349,1398.672358,1818.148469,3014.740104,0.048731,-224.972299,123.656889,-9.754534,56.595062,10.443339,...,7.683082,-10.071786,-0.255097,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596,rock


### 3. Кодирование жанров

In [40]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [41]:
genre_list

0      blues
1      blues
2      blues
3      blues
4      blues
       ...  
995     rock
996     rock
997     rock
998     rock
999     rock
Name: label, Length: 1000, dtype: object

In [42]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

## Приведение данных в общий вид для обучения ИНС

In [43]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-2], dtype = float))

In [44]:
data.iloc[:, :-2]

,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19
0,0.349943,1784.420446,2002.650192,3806.485316,0.083066,-113.596742,121.557302,-19.158825,42.351029,-6.376457,...,10.980491,-8.324323,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208
1,0.340983,1529.835316,2038.617579,3548.820207,0.056044,-207.556796,124.006717,8.930562,35.874684,2.916038,...,11.903744,-5.560387,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431
2,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,-13.987036,...,9.706442,-13.123111,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434
3,0.404779,1070.119953,1596.333948,2185.028454,0.033309,-199.431144,150.099218,5.647594,26.871927,1.754463,...,8.149012,-3.196314,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690
4,0.308590,1835.494603,1748.362448,3580.945013,0.101500,-160.266031,126.198800,-35.605448,22.153301,-32.489270,...,1.206804,-13.083820,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.351991,2008.581132,2106.617024,4254.215942,0.089267,-153.632304,109.857258,-23.085707,59.040677,-7.490499,...,15.402502,-12.380283,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717573
996,0.398653,2006.051164,2068.327905,4147.374921,0.097659,-142.424217,116.219780,-32.177077,49.122974,-8.375371,...,10.800343,-12.982495,14.065346,-16.394693,4.152589,-4.350760,3.736455,-10.845639,1.875218,-7.459579
997,0.432103,2077.190361,1926.989678,4030.767293,0.121824,-125.031313,115.194965,-47.993506,52.834532,-13.480977,...,10.113327,-16.048916,10.481961,-17.951917,1.211113,-11.534863,1.844773,-12.847902,3.447424,-12.594178
998,0.362349,1398.672358,1818.148469,3014.740104,0.048731,-224.972299,123.656889,-9.754534,56.595062,10.443339,...,12.796228,-9.000729,7.683082,-10.071786,-0.255097,-5.276486,-2.816289,-4.416438,1.558265,-5.043121


## Разделение выборки на обучающую и тестовую

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Классификация жанров с использованием deep learning Keras

## Построение ИНС

In [53]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [54]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [55]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
800/800 [==============================] - 1s 828us/step - loss: 2.1903 - accuracy: 0.1875
Epoch 2/20
800/800 [==============================] - 0s 17us/step - loss: 1.8745 - accuracy: 0.3500
Epoch 3/20
800/800 [==============================] - 0s 16us/step - loss: 1.6515 - accuracy: 0.3837
Epoch 4/20
800/800 [==============================] - 0s 15us/step - loss: 1.4791 - accuracy: 0.4538
Epoch 5/20
800/800 [==============================] - 0s 15us/step - loss: 1.3327 - accuracy: 0.5387
Epoch 6/20
800/800 [==============================] - 0s 16us/step - loss: 1.2189 - accuracy: 0.5650
Epoch 7/20
800/800 [==============================] - 0s 17us/step - loss: 1.1386 - accuracy: 0.6212
Epoch 8/20
800/800 [==============================] - 0s 17us/step - loss: 1.0644 - accuracy: 0.6500
Epoch 9/20
800/800 [==============================] - 0s 17us/step - loss: 0.9942 - accuracy: 0.6687
Epoch 10/20
800/800 [==============================] - 0s 16us/step - loss: 0.9480 - accur

In [56]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 199us/step


In [57]:
print('test_acc: ',test_acc)

test_acc:  0.6449999809265137


## Проверка ИНС

In [58]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [59]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 436us/step - loss: 2.2988 - accuracy: 0.0850 - val_loss: 2.1715 - val_accuracy: 0.2850
Epoch 2/30
600/600 [==============================] - 0s 28us/step - loss: 2.1115 - accuracy: 0.3233 - val_loss: 2.0529 - val_accuracy: 0.3000
Epoch 3/30
600/600 [==============================] - 0s 28us/step - loss: 1.9531 - accuracy: 0.3450 - val_loss: 1.9383 - val_accuracy: 0.2950
Epoch 4/30
600/600 [==============================] - 0s 30us/step - loss: 1.8115 - accuracy: 0.3433 - val_loss: 1.8365 - val_accuracy: 0.3300
Epoch 5/30
600/600 [==============================] - 0s 27us/step - loss: 1.6888 - accuracy: 0.3683 - val_loss: 1.7316 - val_accuracy: 0.3700
Epoch 6/30
600/600 [==============================] - 0s 27us/step - loss: 1.5687 - accuracy: 0.4400 - val_loss: 1.6380 - val_accuracy: 0.4100
Epoch 7/30
600/600 [==============================] - 0s 28us/step - loss: 1.4668 - accuracy: 0

In [60]:
results

[1.2284379291534424, 0.625]

## Предсказания

In [62]:
predictions = model.predict(X_test)
predictions

array([[2.7007700e-04, 1.4077120e-04, 4.1779539e-01, ..., 4.0978104e-01,
        5.0197877e-03, 5.4033674e-02],
       [4.8765284e-01, 1.5790272e-05, 1.5746483e-01, ..., 1.3703814e-05,
        1.8099938e-02, 3.1288320e-01],
       [1.6417694e-01, 2.8000276e-03, 5.6183234e-02, ..., 3.5792441e-04,
        5.2364737e-01, 1.6981214e-01],
       ...,
       [7.7783596e-03, 1.4416895e-04, 4.6763915e-04, ..., 4.6710316e-05,
        3.9627563e-04, 7.1784176e-02],
       [2.1683003e-03, 2.8361453e-02, 7.0087373e-02, ..., 5.1980831e-02,
        1.3177706e-01, 4.0159878e-01],
       [9.5311673e-03, 6.0946517e-05, 4.4957320e-05, ..., 2.1202973e-06,
        3.3733429e-04, 2.8691228e-02]], dtype=float32)

In [65]:
np.sum(predictions[0])

1.0

In [69]:
np.argmax(predictions[0])

2

# Сохранение нейросети

In [67]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Загрузка нейросети

In [1]:
from keras.models import model_from_json

jfile = open("model.json", "r")
loaded_model = jfile.read()
jfile.close()
loaded_model = model_from_json(loaded_model)
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.
D:\Anaconda3\envs\introml\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\introml\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\introml\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\introml\lib\site-packages\tensorflow\python\framework\dtype

Instructions for updating:
Colocations handled automatically by placer.


## Конвертация .midi to .wav

In [11]:
from midi2audio import FluidSynth
FluidSynth().midi_to_audio('C:/Users/Яна/MGC/MGC/m1.mid', 'output.wav')

FileNotFoundError: [WinError 2] Не удается найти указанный файл